# import bilbliotek

In [37]:
import os
import json
import requests
from bs4 import BeautifulSoup

## pobieranie opini o konkretnym produkcie

In [22]:
with open("headers.json","r") as file:
    headers = json.load(file)

In [40]:
product_id = "84514582"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
while next_page:
    response = requests.get(next_page, headers=headers)
   #print(response.text)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        opinions = page_dom.select(".js_product-review:not(.user-post--highlight)")
        print(len(opinions))
        for opinion in opinions:
            try: 
                single_opinion = {
                    "review_id": opinion['data-entry-id'],
                    "author":opinion.select_one(".user-post__author-name").text.strip(),
                    "recomendation":opinion.select_one(".user-post__author-recomendation > em").text.strip(),
                    "stars":opinion.select_one(".user-post__score-count").text.strip(),
                    "content":opinion.select_one(".user-post__text").text.strip(),
                    "pros":[p.text.strip() for p in opinion.select("review-feature__item review-feature__item--positive")],
                    "cons":[c.text.strip() for c in  opinion.select("review-feature__item review-feature__item--negative")],
                    "likes":opinion.select_one(".vote-yes > span").text.strip(),
                    "dislikes":opinion.select_one(".vote-no > span").text.strip(),
                    "publish_date":opinion.select_one(".user-post__published > time:nth-child(1)")['datetime'].strip(),
                    "purchase_date":opinion.select_one(".user-post__published > time:nth-child(2)")['datetime'].strip()

                }

                all_opinions.append(single_opinion)

            except (AttributeError, TypeError):
                pass
        try:
            next_page = "https://www.ceneo.pl"+ page_dom.select_one(".pagination__next")["href"]
            
        except TypeError:
            next_page = None
print(all_opinions)

10
10
10
10
10
10
10
10
6
[{'review_id': '13551598', 'author': 'b...a', 'recomendation': 'Polecam', 'stars': '4/5', 'content': 'Wykonana dobrze, trochę za duża, z ledwością mieści mi się na biurku, ale może być. Drukuje ciszej w porównaniu z poprzednią drukarką HP 209a. Napełniłam tusze do pełna i po ok.10 kartkach ubyło mi 4 mm czarnego tuszu w pojemniku. Mam nadzieję, że będzie wydajna. Na razie nie mogę tego stwierdzić. Fajnie, że można zobaczyć, ile ubyło tuszu. Dolewanie jest banalnie proste, tusz nie wylewa się. Martwi mnie tylko, że przenosić trzeba ją w pozycji poziomej, bo inaczej tusz może się wylewać. Ogólnie jestem zadowolona z zakupu.', 'pros': [], 'cons': [], 'likes': '3', 'dislikes': '3', 'publish_date': '2020-12-10 19:41:04', 'purchase_date': '2020-12-04 21:02:01'}, {'review_id': '18299406', 'author': 'U...o', 'recomendation': 'Polecam', 'stars': '4,5/5', 'content': 'Po 1 dniu uzytkowania mozna stwierdzic 1-polaczenie z kompem poprzez wi fi-bez problemu\n2-tankowanie zb

## Zapisanie wszystkich opini o konkretnym produkcie w bazie danych

In [41]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [45]:
with open(f"./opinions/{product_id}.json" , "w", encoding="utf-8") as jf:
    json.dump(all_opinions, jf, indent = 4, ensure_ascii=False)